In [2]:
!pip install ezdxf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np, ezdxf
from pathlib import Path
from math import hypot, sqrt

# ----------------------------------------------------------------------
# -------------------------------------------------------------------------
INPUT_DXF  = Path("contour_updated.dxf")          # UTM‑based source
OUTPUT_DXF = Path("contours_local.dxf")  

# 4 control points  (UTM → Local)
utm   = np.array([[384717.299, 7729358.710],
                  [383922.154, 7730392.702],
                  [384612.840, 7722383.778],
                  [384740.885, 7723557.596]])
local = np.array([[38036.125, 183993.018],
                  [37175.630, 184973.869],
                  [38382.026, 177023.592],
                  [38434.103, 178203.619]])

# ----------------------------------------------------------------------
# 2.  solve affine (Local = A·UTM + t)
rows_A, rows_B = [], []
for (ux, uy), (lx, ly) in zip(utm, local):
    rows_A.extend([[ux, uy, 1, 0, 0, 0],
                   [0, 0, 0, ux, uy, 1]])
    rows_B.extend([lx, ly])

a, b, c, d, e, f = np.linalg.lstsq(
    np.asarray(rows_A, float),
    np.asarray(rows_B, float),
    rcond=None,
)[0]

def to_local(x, y):
    return a*x + b*y + c, d*x + e*y + f

print("Affine:")
print(f"  Local_X = {a:.10f}*UTM_X + {b:.10f}*UTM_Y + {c:.3f}")
print(f"  Local_Y = {d:.10f}*UTM_X + {e:.10f}*UTM_Y + {f:.3f}")

# sanity check
err = np.array([to_local(*p) for p in utm]) - local
print("RMS residual:", sqrt((err**2).mean()), "m")

# ----------------------------------------------------------------------
# 3.  load DXF & transform
doc = ezdxf.readfile(INPUT_DXF)
msp = doc.modelspace()

def tx(pt):
    x, y, z = pt
    lx, ly = to_local(x, y)
    return (lx, ly, z)

for ent in list(msp):               # safe copy
    t = ent.dxftype()

    if t == "LINE":
        ent.dxf.start = tx(ent.dxf.start)
        ent.dxf.end   = tx(ent.dxf.end)

    elif t in {"POINT", "CIRCLE", "ARC"}:
        ent.dxf.center = tx(ent.dxf.center)

    elif t == "LWPOLYLINE":
        # pull xyz plus widths/bulge, push back with same format
        pts = ent.get_points(format="xyseb")  # x, y, startw, endw, bulge
        new_pts = []
        for x, y, sw, ew, bulge in pts:
            lx, ly = to_local(x, y)
            new_pts.append((lx, ly, sw, ew, bulge))
        ent.set_points(new_pts, format="xyseb")

    elif t == "POLYLINE":           # classic 3D
        for v in ent.vertices():
            v.dxf.location = tx(v.dxf.location)

    # add more elifs (SPLINE, INSERT, …) if your file needs them

print(f"Transformed {len(msp)} entities")

# ----------------------------------------------------------------------
# 4.  save
doc.saveas(OUTPUT_DXF)
print("✓ DXF written →", OUTPUT_DXF.resolve())


Affine:
  Local_X = 0.9982610214*UTM_X + -0.0645429578*UTM_Y + 152863.517
  Local_Y = 0.0645496562*UTM_X + 0.9982436778*UTM_Y + -7556623.816
RMS residual: 0.004482897257639093 m
Transformed 2506 entities
✓ DXF written → C:\Users\cesar\OneDrive\Desktop\proyectos python\Rosmery\contours_local.dxf
